准备文件: 找一个 .txt 文件（红楼梦片段、你的小说、或者任何你想要微调的文本）。

上传: 点击 Colab 左侧侧边栏的 “文件夹图标” (Files) -> 点击上传按钮 (或直接拖拽文件进去)。

重命名: 为了代码通用，请在左侧把刚上传的文件重命名为 raw_data.txt。

执行下面代码

In [ ]:
!pip install -q datasets openai

import os
import json
import random
import shutil
from google.colab import drive
from google.colab import userdata
from openai import OpenAI
from datasets import load_dataset
from tqdm import tqdm

# ==========================================
# 1. 基础设施：连接 Google Drive
# ==========================================
print(">>> [系统] 正在挂载 Google Drive...")
drive.mount('/content/drive')

# 最终保存路径 (直接存网盘，防丢失)
DRIVE_SAVE_PATH = "/content/drive/My Drive/project_pocket_mind_train.jsonl"
LOCAL_SRC_FILE = "raw_data.txt"

# ==========================================
# 2. 策略配置
# ==========================================
# 私有数据复制倍数
OVERSAMPLE_RATE = 10
# 引入开源合成数据条数
PUBLIC_DATA_COUNT = 100

# ==========================================
# 3. 核心生产线
# ==========================================
def run_factory():
    # --- A. 检查原料 ---
    if not os.path.exists(LOCAL_SRC_FILE):
        print(f"[错误] 找不到 '{LOCAL_SRC_FILE}'。虚拟机可能已重置。")
        print(f"[提示] 请在左侧 Files 面板重新上传 raw_data.txt！")
        return

    # --- B. 生产私有数据 (DeepSeek) ---
    try:
        API_KEY = userdata.get('DEEPSEEK_API_KEY')
    except:
        # 如果未配置 Secrets，请在此处填入 Key
        API_KEY = "你的_DEEPSEEK_API_KEY_填在这里"

    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

    print(">>> [步骤 1/3] DeepSeek 正在蒸馏私有数据...")
    with open(LOCAL_SRC_FILE, "r", encoding="utf-8") as f:
        text = f.read()
        # 简单切分前 5 段
        chunks = [text[i:i+600] for i in range(0, len(text), 600)][:5]

    private_data = []
    for chunk in tqdm(chunks):
        try:
            resp = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "生成带<think>标签的JSON问答: {\"instruction\":..., \"output\": \"<think>...</think>...\"}"},
                    {"role": "user", "content": f"内容:\n{chunk}"}
                ],
                response_format={"type": "json_object"}
            )
            item = json.loads(resp.choices[0].message.content)
            if "<think>" in item.get("output", ""):
                item["source"] = "private"
                private_data.append(item)
        except Exception as e:
            print(f"  [警告] 跳过: {e}")

    if not private_data:
        print("[错误] 私有数据生成失败，请检查 API Key。")
        return

    # --- C. 引入合成数据 (Hugging Face) ---
    print(f">>> [步骤 2/3] 正在混合开源合成数据 (Bespoke-Stratos)...")
    public_data = []
    try:
        # 使用社区高质量的 R1 复刻数据集
        ds = load_dataset("bespokelabs/Bespoke-Stratos-17k", split="train", streaming=True)
        for item in ds.take(PUBLIC_DATA_COUNT):
            if 'conversations' in item:
                public_data.append({
                    "instruction": item['conversations'][0]['value'],
                    "input": "",
                    "output": item['conversations'][1]['value'],
                    "source": "synthetic_public"
                })
    except Exception as e:
        print(f"[警告] 连接 HF 失败 ({e})，将仅使用私有数据。")

    # --- D. 混合与打散 ---
    print(f">>> [步骤 3/3] 执行混合增强 (私有数据 x{OVERSAMPLE_RATE})...")

    # 核心配方：(私有数据 * 10) + 合成数据
    final_dataset = (private_data * OVERSAMPLE_RATE) + public_data
    random.shuffle(final_dataset) # 打散数据

    # --- E. 落盘保存 ---
    print(f">>> [系统] 正在写入 Google Drive: {DRIVE_SAVE_PATH}")
    with open(DRIVE_SAVE_PATH, "w", encoding="utf-8") as f:
        for entry in final_dataset:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print("\n==============================================")
    print(f"[完成] 数据工厂任务圆满完成！")
    print(f"数据统计:")
    print(f"   - 私有数据 (原始): {len(private_data)} 条")
    print(f"   - 私有数据 (增强后): {len(private_data) * OVERSAMPLE_RATE} 条")
    print(f"   - 合成数据 (开源): {len(public_data)} 条")
    print(f"   - 总训练样本: {len(final_dataset)} 条")
    print(f"文件已永久保存至网盘。")
    print("==============================================")

# 执行
if __name__ == "__main__":
    run_factory()

>>> [系统] 正在挂载 Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> [步骤 1/3] DeepSeek 正在蒸馏私有数据...


 20%|██        | 1/5 [00:17<01:10, 17.67s/it]

  [警告] 跳过: Unterminated string starting at: line 3 column 13 (char 75)


100%|██████████| 5/5 [01:43<00:00, 20.63s/it]


>>> [步骤 2/3] 正在混合开源合成数据 (Bespoke-Stratos)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

>>> [步骤 3/3] 执行混合增强 (私有数据 x10)...
>>> [系统] 正在写入 Google Drive: /content/drive/My Drive/project_pocket_mind_train.jsonl

[完成] 数据工厂任务圆满完成！
数据统计:
   - 私有数据 (原始): 4 条
   - 私有数据 (增强后): 40 条
   - 合成数据 (开源): 100 条
   - 总训练样本: 140 条
文件已永久保存至网盘。


In [ ]:
import torch
from google.colab import output

print(">>> 🚀 正在初始化 Unsloth (二进制极速版)...")

# 1. 强制安装 Unsloth 的最新适配版
# --no-cache-dir 防止它去读刚才编译失败的缓存
!pip install --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. 安装依赖 (不锁定版本，让它自己找最新的 whl 包)
!pip install --no-deps xformers trl peft accelerate bitsandbytes

output.clear()
print(">>> ✅ 环境安装成功！")

# 检查一下是不是真的成功了
if torch.cuda.is_available():
    print(f">>> GPU: {torch.cuda.get_device_name(0)}")
    print(f">>> PyTorch: {torch.__version__}")
else:
    print(">>> ❌ 坏了，GPU 没挂载上，请检查 Runtime 设置！")

>>> ✅ 环境安装成功！
>>> GPU: Tesla T4
>>> PyTorch: 2.9.0+cu126


加载模型: Llama-3.2-3B-Instruct (4-bit 量化版)。

挂载 LoRA: 配置适配器。

加载数据: 读取刚才生成的 project_pocket_mind_train.jsonl

In [ ]:
from unsloth import FastLanguageModel
import torch
import os
from google.colab import drive
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# ==========================================
# 0. 基础设施修复：连接数据仓库
# ==========================================
# 确保网盘挂载，防止路径访问失败
drive.mount('/content/drive')

# 🎯 关键修正：指定网盘绝对路径
DATA_PATH = "/content/drive/My Drive/project_pocket_mind_train.jsonl"

# 做一个工程检查，避免瞎跑
if not os.path.exists(DATA_PATH):
    print(f"❌ 严重错误: 在网盘中未找到数据: {DATA_PATH}")
    print("👉 请检查上一张是否执行了“直写网盘”脚本，或者文件名是否正确。")
    # 尝试回退到本地临时文件(如果刚才没重启过)
    if os.path.exists("project_pocket_mind_train.jsonl"):
        print("⚠️ 发现本地临时文件，降级使用本地文件...")
        DATA_PATH = "project_pocket_mind_train.jsonl"
    else:
        raise FileNotFoundError("找不到训练数据！")
else:
    print(f"✅ 数据路径确认有效: {DATA_PATH}")

# ==========================================
# 1. 模型加载 (Engine Start)
# ==========================================
max_seq_length = 2048
dtype = None
load_in_4bit = True

print(">>> 正在加载 Llama-3.2-3B-Instruct 模型...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# ==========================================
# 2. 配置 LoRA 适配器
# ==========================================
print(">>> 正在挂载 LoRA 适配器...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# ==========================================
# 3. 数据格式化 (Data Formatting)
# ==========================================
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# 🎯 关键修正：这里使用 DATA_PATH
print(f">>> 正在读取数据: {DATA_PATH}")
dataset = load_dataset("json", data_files=DATA_PATH, split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# ==========================================
# 4. 训练参数配置
# ==========================================
print(">>> 🚀 启动训练引擎...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# ==========================================
# 5. 执行训练
# ==========================================
trainer_stats = trainer.train()

print(f"\n>>> ✅ 训练完成！耗时: {trainer_stats.metrics['train_runtime']:.2f} 秒")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 数据路径确认有效: /content/drive/My Drive/project_pocket_mind_train.jsonl
>>> 正在加载 Llama-3.2-3B-Instruct 模型...
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
>>> 正在挂载 LoRA 适配器...
>>> 正在读取数据: /content/drive/My Drive/project_pocket_mind_train.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/140 [00:00<?, ? examples/s]

>>> 🚀 启动训练引擎...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/140 [00:00<?, ? examples/s]

...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.453100
20,1.202000
30,0.943700
40,0.778100
50,0.663000
60,0.586300


wandb: WARNING URL not available in offline run


train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,▃▁██▅▅
train/learning_rate,█▇▅▄▂▁
train/loss,█▆▄▃▂▁
total_flos,8137080108920832.0
train/epoch,3.34286
train/global_step,60
train/grad_norm,0.25597
train/learning_rate,0.0
train/loss,0.5863



>>> ✅ 训练完成！耗时: 811.17 秒


特意准备了一个红楼梦相关的问题，看看它能不能用“红学专家”的口吻回答

In [ ]:
# ==========================================
# 🔍 验货环节：模型推理 (Inference)
# ==========================================
from transformers import TextStreamer

# 1. 开启推理加速模式 (Unsloth 的原生优化，速度翻倍)
FastLanguageModel.for_inference(model)

# 2. 准备测试题 (根据你的红楼梦数据定制)
# 这个问题考察逻辑分析能力，不仅仅是背诵
test_instruction = "请分析《红楼梦》的深层隐喻。"
input_text = ""

# 3. 包装成训练时的 Alpaca 格式
# ⚠️ 必须和训练时格式一模一样，否则模型会变傻
prompt = alpaca_prompt.format(test_instruction, input_text, "")

# 4. 把文本转成 Token
inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

# 5. 生成回答
# Streamer 会像打字机一样一个字一个字吐出来，非常解压
print(f">>> 🤖 提问: {test_instruction}")
print(">>> 🧠 模型正在思考 (请屏住呼吸，寻找 <think> 标签)...\n")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer = text_streamer,
    max_new_tokens = 1024, # 允许它多写点
    temperature = 0.6,     # 稍微降低一点创造力，让逻辑更严密
    top_p = 0.9,
    use_cache = True
)

>>> 🤖 提问: 请分析《红楼梦》的深层隐喻。
>>> 🧠 模型正在思考 (请屏住呼吸，寻找 <think> 标签)...

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
请分析《红楼梦》的深层隐喻。

### Input:


### Response:
<think>用户提供的文本是《红楼梦》一书的片段，涉及主题是分析《红楼梦》的深层隐喻。我需要从文本中提取相关信息，来回答关于该书隐喻的解释。

首先，文本开始提到《红楼梦》是一部经典之作，包含着多个层级的隐喻。我需要识别出这些层级，例如语言、图像、情节、人物、事件等方面的隐喻，并分析它们的含义。

1. **语言隐喻**: 文本中提到语言是隐喻的基础，例如“情语淫滥，繁华文句淫秽”，“诗僧情话”，“话中含淫”。我需要解释这些语言如何传达隐喻，例如语言的性质、隐喻的类型（如比喻、 metaphor）和其在故事中的作用。

2. **图像隐喻**: 文本中提到空空道人、石头记、苏轼文忠record等图像作为隐喻。我需要分析这些图像的意义，例如它们代表什么（如道德原则、伦常）和如何在故事中运用。

3. **情节隐喻**: 文本中谈到西子、子建、潘安、子建再次遇子建、石头记录等情节。我需要解释这些情节的隐喻内容，例如传达伦常、批评奸淫或促使Character发展。

4. **人物隐喻**: 文本中人物如空空道人、石头记、苏轼、子建、潘安、子建、西子、清良、英僧、英僧妻等。我需要分析这些人物的角色和隐喻含义，例如他们代表什么（如道德品质）和在故事中的作用。

5. **事件隐喻**: 文本中涉及各种事件如空空道人与石头记合作、石头记录西子之风等。我需要解释这些事件的隐喻含义，例如传达伦常或促使Character发展。

总结来分析：隐喻主要通过语言、图像、情节、人物和事件组合来传达。语言强调淫秽性质，但以隐喻方式传达；图像如空空道人、石头记、苏轼等代表伦常原则；情节如西子、子建等传

训练完立刻存盘

In [ ]:
# ==========================================
# 💾 最终环节：模型保存 (Save to Drive)
# ==========================================
import os
from google.colab import drive

# 1. 确保网盘连接
drive.mount('/content/drive')

# 2. 定义保存路径 (存到网盘里，永久不丢)
# 我们把它命名为 "Pocket_Mind_Llama3_LoRA"
OUTPUT_DIR = "/content/drive/My Drive/Pocket_Mind_Llama3_LoRA"

print(f">>> 💾 正在将模型权重保存至: {OUTPUT_DIR} ...")

# 3. 保存 LoRA 适配器 (只保存我们训练的那 0.7% 的参数，非常小，可能只有几百MB)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# 4. 验证是否保存成功
if os.path.exists(f"{OUTPUT_DIR}/adapter_model.safetensors"):
    print("\n✅ 恭喜！保存成功！")
    print(f"📁 你的专属模型已安全存储在 Google Drive 中。")
    print("   主要文件: adapter_model.safetensors (你的训练成果)")
else:
    print("\n❌ 保存似乎失败了，请检查网盘空间或路径。")

# ==========================================
# 🔮 (可选预览) 下一步预告：GGUF 转换
# ==========================================
print("\n👉 下一步提示：")
print("现在的模型是 'LoRA 格式'。如果你想把它装进手机 (Ollama) 或电脑本地跑，")
print("我们需要执行 'GGUF 格式转换'。")
print("如果你准备好了，请告诉我 '开始转换 GGUF'。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> 💾 正在将模型权重保存至: /content/drive/My Drive/Pocket_Mind_Llama3_LoRA ...

✅ 恭喜！保存成功！
📁 你的专属模型已安全存储在 Google Drive 中。
   主要文件: adapter_model.safetensors (你的训练成果)

👉 下一步提示：
现在的模型是 'LoRA 格式'。如果你想把它装进手机 (Ollama) 或电脑本地跑，
我们需要执行 'GGUF 格式转换'。
如果你准备好了，请告诉我 '开始转换 GGUF'。


加载基础模型

In [5]:
from unsloth import FastLanguageModel
import torch
from google.colab import drive

# 1. 确保网盘连接
drive.mount('/content/drive')

# 2. 定义参数
max_seq_length = 2048
dtype = None
load_in_4bit = True
LORA_PATH = "/content/drive/My Drive/Pocket_Mind_Llama3_LoRA"

print(f">>> 🔄 正在修正并重新加载 LoRA: {LORA_PATH}")

# 🔥【关键修正点】这里必须用 "model, tokenizer =" 来接收，把包裹拆开！
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LORA_PATH,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ 模型本体已提取！现在 'model' 变量是真正的模型对象了。")
print("👉 请立即重新运行下面的 'GGUF 转换' 代码块，这次一定能行！")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> 🔄 正在修正并重新加载 LoRA: /content/drive/My Drive/Pocket_Mind_Llama3_LoRA
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ 模型本体已提取！现在 'model' 变量是真正的模型对象了。
👉 请立即重新运行下面的 'GGUF 转换' 代码块，这次一定能行！


In [4]:
# ==========================================
# 📦 最终工序：GGUF 格式打包 (Export to GGUF)
# ==========================================
import os

# 1. 定义保存路径 (直接存网盘)
# 我们存两个版本：
# - q4_k_m: 4-bit 量化版 (体积小，速度快，推荐手机/笔记本用)
# - f16:    半精度版 (体积大，精度最高，适合服务器用)
SAVE_DIR = "/content/drive/My Drive/Pocket_Mind_GGUF"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

print(f">>> 🏭 正在开始 GGUF 转换流水线...")
print(f">>> 目标路径: {SAVE_DIR}")

# 2. 转换并保存 (Unsloth 内置了 llama.cpp 引擎，非常方便)

# --- 版本 A: 推荐版 (q4_k_m) ---
print("\n>>> [1/2] 正在生成 4-bit 量化版 (q4_k_m) ...")
print("    (这个版本适合 Ollama 和绝大多数电脑，体积约 2GB)")
try:
    model.save_pretrained_gguf(
        SAVE_DIR,
        tokenizer,
        quantization_method = "q4_k_m"
    )
    print("✅ q4_k_m 版本转换完成！")
except Exception as e:
    print(f"❌ q4_k_m 转换失败: {e}")

# --- 版本 B: 高清版 (f16) ---
# 如果你网盘空间不够(需要6GB)，可以注释掉下面这段
# print("\n>>> [2/2] 正在生成 FP16 高清版 (f16) ...")
# try:
#     model.save_pretrained_gguf(
#         SAVE_DIR,
#         tokenizer,
#         quantization_method = "f16"
#     )
#     print("✅ f16 版本转换完成！")
# except Exception as e:
#     print(f"❌ f16 转换失败: {e}")

print("\n==============================================")
print(f"🎉 全部工序完成！你的模型已准备好部署。")
print(f"📂 请去 Google Drive 查看: {SAVE_DIR}")
print("==============================================")

>>> 🏭 正在开始 GGUF 转换流水线...
>>> 目标路径: /content/drive/My Drive/Pocket_Mind_GGUF

>>> [1/2] 正在生成 4-bit 量化版 (q4_k_m) ...
    (这个版本适合 Ollama 和绝大多数电脑，体积约 2GB)
❌ q4_k_m 转换失败: 'tuple' object has no attribute 'save_pretrained_gguf'

🎉 全部工序完成！你的模型已准备好部署。
📂 请去 Google Drive 查看: /content/drive/My Drive/Pocket_Mind_GGUF
